<a href="https://colab.research.google.com/github/krisdanner/Fun-o-de-Convers-o-nica/blob/main/Fun%C3%A7%C3%A3o_de_Convers%C3%A3o_%C3%9Anica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import openpyxl
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime, timezone, timedelta

data_e_hora_i = datetime.now().astimezone(timezone(timedelta(hours=-3))).strftime('%d/%m/%Y %H:%M')

url = 'https://data-live.flightradar24.com/zones/fcgi/feed.js?faa=1&bounds=0.977%2C-5.651%2C-57.118%2C-27.104&satellite=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=14400&gliders=1&stats=1'

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}

r = requests.get(url, headers=headers)
data = json.loads(r.content)

codigo = []
aviao = []
dados = []
dados3 = []

def geo2aer(lat, lon, h, lat0, lon0, h0):
  dados3.append(lat); dados3.append(lon); dados3.append(h)
  a = 6378137
  e = 0.0818191908426215
  lat = math.radians(lat)
  lon = math.radians(lon)
  lat0 = math.radians(lat0)
  lon0 = math.radians(lon0)

  N = a / math.sqrt(1 - e**2 * math.sin(lat)**2)
  N0 = a / math.sqrt(1 - e**2 * math.sin(lat0)**2)

  X = (N+h)*math.cos(lat)*math.cos(lon)
  Y = (N+h)*math.cos(lat)*math.sin(lon)
  Z = (N*(1 - e**2)+h)*math.sin(lat)

  X0 = (N0+h0)*math.cos(lat0)*math.cos(lon0)
  Y0 = (N0+h0)*math.cos(lat0)*math.sin(lon0)
  Z0 = (N0*(1-e**2)+h0)*math.sin(lat0)

  A = [[-math.sin(lon0), math.cos(lon0), 0], [-math.sin(lat0)*math.cos(lon0), -math.sin(lat0)*math.sin(lon0), math.cos(lat0)], [math.cos(lat0)*math.cos(lon0), math.cos(lat0)*math.sin(lon0), math.sin(lat0)]]
  B = [[X-X0], [Y-Y0], [Z-Z0]]
  A = np.dot(A,B)
  x = A[0][0]
  y = A[1][0]
  z = A[2][0]

  if y > 0:
    az = math.degrees(math.atan(math.radians(x)/math.radians(y)))
  if y < 0:
    az = math.degrees(math.atan(math.radians(x)/math.radians(y)))+180
  if y == 0 and x > 0:
    az = 90
  if y == 0 and x < 0:
    az = 270
  if az < 0:
    az = az+360

  x = math.radians(x)
  z = math.radians(z)
  el = math.atan((z*math.sin(math.radians(az)))/x)
  dist = z/math.sin(el)
  el = math.degrees(el)
  dist = math.degrees(dist)
  return [round(az, 3),round(el, 3),round(dist, 3)]

for x in data:
    codigo.append(x)

del codigo[0], codigo[0]
codigo.pop()

for x in codigo:
    dados.append(round((data[x][4])*0.3048, 2))
    dados.append(data[x][1])
    dados.append(data[x][2])
    aviao.append(data[x][16])

col = 'Altitude Latitude Longitude'.split()
lin = aviao
dados_1 = np.array(dados).reshape(len(aviao), 3)
dataf = pd.DataFrame(data=dados_1, index=lin, columns=col)
print(dataf)

pos = int(input('Digite a posição da aeronave: '))
qtd = int(input('Digite quantos dados de posição deseja obter: '))

t = 0

while t < qtd:
    r = requests.get(url, headers=headers)
    data = json.loads(r.content)
    dados2 = []

    dados2.append((data[codigo[pos]][4])*0.3048)
    dados2.append(data[codigo[pos]][1])
    dados2.append(data[codigo[pos]][2])

    aer = []
    aer = geo2aer(dados2[1],dados2[2],dados2[0],-2.3310835100,-44.4208387800,58.826)
    print(round(100*t/qtd, 2),'% ',aer)

    #print(round(100 * t / qtd, 2), '%')
    t += 1

hora_f = datetime.now().astimezone(timezone(timedelta(hours=-3))).strftime('%H:%M')

col = 'Latitude Longitude Altitude'.split()
dados_2 = np.array(dados3).reshape(qtd, 3)
dataf = pd.DataFrame(data=dados_2, columns=col)
dataf.to_excel('Dados_de_localização_GEO '+aviao[pos]+'.xlsx')

plt.style.use('fast')
figura = plt.figure(figsize=(12,8))
grafico = figura.gca(projection='3d')

plt.plot(dataf['Longitude'], dataf['Latitude'], dataf['Altitude'], color='red')
plt.xlabel('LONGITUDE', color='blue', fontsize=15, weight='bold')
plt.ylabel('LATITUDE', color='blue', fontsize=15, weight='bold')
plt.title('Voo Aeronave: '+aviao[pos]+' '+data_e_hora_i+'-'+hora_f, fontsize=20)
plt.savefig('Gráfico '+aviao[pos]+'.png', dpi=300)
plt.show()
